# Calibración de sensores

Los sensores de bajo costo suelen entregar como salida una señal analógica que ha de interpretarse según la magnitud que se desea medir. Para interpretar esta cantidad, los sensores analógicos se calibran utilizando una referencia de magnitud ya conocida, o utilizando cantidades conocidas según el sensor.

En general, se suele hacer una interpolación lineal de puntos *(voltaje, magnitud)*. Para el caso de un ajuste lineal de dos puntos $(x_1, y_1)$ y $(x_2, y_2)$, basta encontrar los valores $m$ y $b$ de la ecuación $y = m\cdot x + b$. Para esto se calcula
\begin{align}
    m &= \frac{y_2 - y_1}{x_2 - x_1} \\
    b &= y_2 - m\cdot x_2
      &= y_1 - m\cdot x_1
\end{align}

Aquí se reportan los fundamentos algebraicos de los procedimientos de calibración de algunos sensores análogos, siguiendo como referencia la implementación del fabricante y una reimplementación propia.

## 1. pH - *Gravity: Analog pH Sensor / Meter Kit V2*
- **Implementación de referencia en el repositorio [niclabs/DFRobot_PH](https://github.com/niclabs/DFRobot_PH).**
- **Descripción del producto en https://wiki.dfrobot.com/Gravity__Analog_pH_Sensor_Meter_Kit_V2_SKU_SEN0161-V2**

Se reporta en otro sensor de alto costo ([Atlas Scientific](https://www.atlas-scientific.com/files/pH_EZO_Datasheet.pdf)) el uso de una calibración en 3 puntos, en soluciones de referencia de pH 4, 7 y 10. Sin embargo, la librería original de DFRobot sólo se calibra en dos puntos, pH 4 y pH 7.

El valor de pH entregado viene dado por el ajuste lineal de los puntos de calibración (4, $v_\text{pH 4}$) y (7, $v_\text{pH 7}$).

## 2. Conductividad eléctrica - *Gravity: Analog TDS Sensor / Meter For Arduino*
- **Implementación de referencia en el repositorio [niclabs/GravityTDS](https://github.com/niclabs/GravityTDS).**
- **Descripción del producto en https://wiki.dfrobot.com/Gravity__Analog_TDS_Sensor___Meter_For_Arduino_SKU__SEN0244**

El sensor Gravity TDS está diseñado para entregar un valor de Total de sólidos disueltos (*Total dissolved solids*) en partes por millón \[ppm\] calculado a partir un valor de conductividad eléctrica en \[μS/cm\], siguiendo la relación:
$$
    \text{TDS [ppm]} = \frac{\text{EC [μS/cm]}}{2}
$$

En la implementación, el valor de conductividad eléctrica a 25 °C se interpreta en función del voltaje de la señal de salida con la fórmula:
$$
    \text{EC}(v) = K\cdot(133.42\cdot v^3 - 255.86\cdot v^2 + 857.39\cdot v)
$$
Como la temperatura puede alejarse de los 25 °C, se realiza una compensación para una temperatura $t$ cualquiera en celsius, con la fórmula:
$$
    \text{EC}_\text{compensada}(v, t) =  \text{EC}(v)\cdot(1+0.02\cdot(t-25))
$$

Para obtener el valor de $K$, es necesario una solución de referencia de conductividad eléctrica conocida, como puede serlo 1413 \[μS/cm\]. Dado el valor $S$ de conductividad eléctrica conocida, $v_S$ el voltaje entregado por el sensor al medirlo, y $t_S$ la temperatura a la que se midió, el procedimiento es el siguiente.
1. Se compensa $S$ por temperatura, obteniendo $S_\text{compensado} \gets S\cdot(1+0.02\cdot(t_S-25))$.
2. Se calcula $K \gets \frac{S_\text{compensado}}{133.42\cdot v_S^3 - 255.86\cdot v_S^2 + 857.39\cdot v_S}$

Así, se puede condensar todo el procedimiento reescribiendo la fórmula de conductividad eléctrica con compensación de temperatura
$$
    \text{EC}_\text{compensada}(v, t) =  S\cdot(1+0.02\cdot(t_S-25))\frac{133.42\cdot v^3 - 255.86\cdot v^2 + 857.39\cdot v}{133.42\cdot v_S^3 - 255.86\cdot v_S^2 + 857.39\cdot v_S}\cdot(1+0.02\cdot(t-25))
$$

## 3. Turbidez - *Gravity: Analog Turbidity Sensor For Arduino*
- **Implementación de referencia en el repositorio [niclabs/GravityTurbidity](https://github.com/niclabs/GravityTurbidity)**
- **Descripción del producto en https://wiki.dfrobot.com/Turbidity_sensor_SKU__SEN0189**

El sensor de turbidez Gravity está diseñado para entregar valores de turbidez entre 0 y 3000 NTU, según se desprende de lo [reportado por el fabricante](https://wiki.dfrobot.com/Turbidity_sensor_SKU__SEN0189#target_4). Si bien el fabricante entrega la fórmula para interpretar turbidez dado un voltaje $v$:
$$
    \text{turb}_\text{fabricante}(v) \text{ [NTU]} = -1120.4\cdot v^2 + 5742.3\cdot v - 4352.9
$$
nuestros experimentos muestran que una parábola no es la curva de mejor ajuste. Nosotros proponemos la fórmula
$$
    \text{turb}(v) \text{ [NTU]} = A e^{\alpha v}
$$
con parámetros $A$ y $\alpha$ por determinar en la calibración.

En el procedimiento toman al menos dos puntos de calibración $(\text{NTU}, v)$, y se realiza una transformación semilogarítmica para poder realizar un ajuste lineal:
\begin{align}
    \text{NTU}(v) &= A e^{\alpha v} && \ln \text{ a ambos lados}\\
    \underbrace{\ln(\text{NTU}(v))}_y &= \underbrace{\ln(A)}_b + \underbrace{\alpha}_m\cdot v
\end{align}

Así, una vez realizado el ajuste de los puntos $(\ln(\text{NTU}), v)$ y obtenido los coeficientes $m$ y $b$, se pueden determinar los parámetros $A$ y $\alpha$ 
\begin{align}
    A &\gets e^{b}\\
    \alpha &\gets m
\end{align}